In [1]:
!pip install datasets evaluate transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 447.3 kB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=b746966d5db19fe89666d600aac928c565ec306dac46fe6f494b1289f44f8d94
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd "/content/drive/MyDrive/week-4"

/content/drive/.shortcut-targets-by-id/1wCkFBZu8tASt7CdIfbiROqkWPsI1jC3g/week-4


In [12]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer
)
import evaluate
import numpy as np
import pandas as pd

In [6]:
# Load the CoNLL-formatted file into memory
def load_conll_data(filepath):
    sentences = []
    current_tokens = []
    current_labels = []
    with open(filepath, encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line:  # blank line indicates end of a sentence
                if current_tokens:
                    sentences.append((current_tokens, current_labels))
                current_tokens, current_labels = [], []
            else:
                token, label = line.split(maxsplit=1)
                current_tokens.append(token)
                current_labels.append(label)
    print(f"Loaded {len(sentences)} labeled sentences")
    return sentences

sentences = load_conll_data("labeled_telegram_product_price_location_cleaned.conll")
# Split into train/validation sets (80/20 split)
train_sentences, val_sentences = train_test_split(sentences, test_size=0.2, random_state=42)
len(train_sentences), len(val_sentences)


Loaded 174645 labeled sentences


(139716, 34929)

In [7]:
from transformers import AutoTokenizer

# Choose a pre-trained model tokenizer (XLM-RoBERTa base for multilingual support)
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# Define label mappings
unique_labels = set(label for _, labels in sentences for label in labels) # Use all sentences to find unique labels
label_to_id = {label: idx for idx, label in enumerate(sorted(unique_labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}
num_labels = len(label_to_id)
print(unique_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

{'O'}


In [10]:
def tokenize_and_align_labels(examples):
    # 1) Tokenize all sentences in this batch at once
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    # 2) Grab the NER labels from the batch
    all_labels = examples["labels"]  # or "ner_tags"
    new_labels = []

    # 3) For each sentence in the batch, build its aligned label list
    for i, labels in enumerate(all_labels):
        word_ids    = tokenized_inputs.word_ids(batch_index=i)
        prev_word   = None
        label_ids   = []

        for word_idx in word_ids:
            if word_idx is None:
                # special tokens
                label_ids.append(-100)
            elif word_idx != prev_word:
                # first subword of a new word
                label_ids.append(label_to_id[labels[word_idx]])
            else:
                # subsequent subword of the same word
                label_ids.append(-100)
            prev_word = word_idx

        new_labels.append(label_ids)

    # 4) Insert the aligned labels back into the tokenized batch
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

# Create Dataset objects from the loaded data
train_data = Dataset.from_dict({"tokens": [s[0] for s in train_sentences], "labels": [s[1] for s in train_sentences]})
val_data = Dataset.from_dict({"tokens": [s[0] for s in val_sentences], "labels": [s[1] for s in val_sentences]})

## Fine-Tuning the Transformer Model

In [18]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification # Import DataCollator
)
import evaluate
import numpy as np
import pandas as pd
import time

def tokenize_and_align_labels(examples):
    # 1) Tokenize all sentences in this batch at once
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        # Remove padding here
    )

    # 2) Grab the NER labels from the batch
    all_labels = examples["labels"]
    new_labels = []

    # 3) For each sentence in the batch, build its aligned label list
    for i, labels in enumerate(all_labels):
        word_ids    = tokenized_inputs.word_ids(batch_index=i)
        prev_word   = None
        label_ids   = []

        for word_idx in word_ids:
            if word_idx is None:
                # special tokens
                label_ids.append(-100)
            elif word_idx != prev_word:
                # first subword of a new word
                label_ids.append(label_to_id[labels[word_idx]])
            else:
                # subsequent subword of the same word
                label_ids.append(-100)
            prev_word = word_idx

        new_labels.append(label_ids)

    # 4) Insert the aligned labels back into the tokenized batch
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

# Define compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = evaluate.load("seqeval").compute(predictions=true_predictions, references=true_labels)
    return {
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


models_to_compare = [
    "xlm-roberta-base",
    "distilbert-base-multilingual-cased",
    "bert-base-multilingual-cased"
]
results = []

for ckpt in models_to_compare:
    # 1) load tokenizer + model
    tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=True)
    model     = AutoModelForTokenClassification.from_pretrained(
                    ckpt,
                    num_labels=num_labels,
                    id2label=id_to_label,
                    label2id=label_to_id
                )

    # 2) retokenize & align
    def align_fn(ex):
        return tokenize_and_align_labels(ex)  # uses current tokenizer, label2id
    tok_train = train_data.map(align_fn, batched=True)
    tok_val   = val_data.map(align_fn, batched=True)

    # 3) training setup
    args = TrainingArguments(
        output_dir=f"./{ckpt}-ner",
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=2,
        weight_decay=0.01,
        logging_steps=50,
        save_steps=1000, # Increased save steps
        save_total_limit=1
    )

    # Define data collator
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tok_train,
        eval_dataset=tok_val,
        tokenizer=tokenizer, # Keep tokenizer for data collator
        data_collator=data_collator, # Add data collator
        compute_metrics=compute_metrics
    )

    # 4) train & evaluate
    start = time.time()
    trainer.train()
    elapsed = time.time() - start
    metrics = trainer.evaluate()

    results.append({
        "model":       ckpt,
        "f1":          metrics["eval_f1"],
        "accuracy":    metrics["eval_accuracy"],
        "train_time_s": round(elapsed, 1)
    })

# 5) compare in a DataFrame
df = pd.DataFrame(results)
print(df)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/139716 [00:00<?, ? examples/s]

Map:   0%|          | 0/34929 [00:00<?, ? examples/s]

/tmp/ipython-input-18-3806594617.py:111: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
!pip install shap lime

from transformers import pipeline
import shap
from lime.lime_text import LimeTextExplainer

# Load your fine-tuned model & tokenizer
ner_pipe = pipeline(
    "ner",
    model="amharic-ner-model",
    tokenizer=tokenizer,
    grouped_entities=True
)

# Sample a few validation messages
sample_texts = [
    "በ200 ብር የሚሸጥ ጥቁር ቡድን በቦሌ አካባቢ ይገኛል።",
    "የቪታሚን ማና በ 1500 ብር ውስጥ አለ።"
]

# SHAP explanation
explainer = shap.Explainer(ner_pipe)
shap_values = explainer(sample_texts)
for i, text in enumerate(sample_texts):
    shap.plots.text(shap_values[i])

# LIME explanation
lime_explainer = LimeTextExplainer(class_names=sorted(id_to_label.values()))
for text in sample_texts:
    exp = lime_explainer.explain_instance(
        text,
        ner_pipe,
        num_features=10
    )
    exp.show_in_notebook(text=True)


In [ ]:
# Load your scraped data
df = pd.read_csv("telegram_data.csv", parse_dates=["Date"])

# If you have a 'Views' column, use it; otherwise simulate:
if "Views" not in df.columns:
    import random
    df["Views"] = df["ID"].apply(lambda x: random.randint(100, 2000))

# 2) Derive metrics per vendor
df["Week"] = df["Date"].dt.to_period("W").apply(lambda r: r.start_time)
posts_per_week = (
    df.groupby(["Channel Username", "Week"])
      .size()
      .groupby("Channel Username")
      .mean()
      .rename("Posts_Per_Week")
)
avg_views = df.groupby("Channel Username")["Views"].mean().rename("Avg_Views")

df["Price_ETB"] = (
    df["Message"]
      .str.extract(r"(\d+)\s*ብር")[0]
      .astype(float)
      .fillna(0)
)
avg_price = df.groupby("Channel Username")["Price_ETB"].mean().rename("Avg_Price")

# Compute a simple lending score
score = (0.5 * avg_views + 0.5 * posts_per_week).rename("Lending_Score")

# Compile the scorecard
scorecard = pd.concat(
    [avg_views, posts_per_week, avg_price, score],
    axis=1
).reset_index().sort_values("Lending_Score", ascending=False)

# Display
scorecard.style.format({
    "Avg_Views": "{:.1f}",
    "Posts_Per_Week": "{:.2f}",
    "Avg_Price": "{:.2f}",
    "Lending_Score": "{:.2f}"
})
